In [1]:
import pandas as pd
import numpy as np
from numpy import genfromtxt
from sklearn.metrics import brier_score_loss
import os
import time
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from joblib import dump, load

In [3]:

year_list = ['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018'] #'2004','2005','2006','2007',
x_axis = np.arange(2004,2019) #2004-2014
model_path = 'log_without_norm_models/2004_logistic.joblib'
feature_data_path = 'without_norm_datasets/test/features/'
target_data_path = 'without_norm_datasets/test/target/'
feature_data_path_train = 'without_norm_datasets/train/features/'
target_data_path_train = 'without_norm_datasets/train/target/'

every_year_list = []
auc_train = []
BSS_per_year = []
DR_list = []
BS_per_year = []
for year in year_list:
    feature_data_file = os.path.join(feature_data_path, '{}_log.csv'.format(year))
    target_data_file = os.path.join(target_data_path, '{}_log.csv'.format(year))
    feature_data_file_train = os.path.join(feature_data_path_train, '{}_log.csv'.format(year))
    target_data_file_train = os.path.join(target_data_path_train, '{}_log.csv'.format(year))
    
    model_this_year = load('log_without_norm_models/{}_logistic.joblib'.format(year)) 
    X = genfromtxt(feature_data_file, delimiter=',')
    y = genfromtxt(target_data_file, delimiter=',')
    X_train = genfromtxt(feature_data_file_train, delimiter=',')
    y_train = genfromtxt(target_data_file_train, delimiter=',')
    
    y_scores = model_this_year.decision_function(X)  
    predict = model_this_year.predict(X)
    
    every_year_list.append(roc_auc_score(y, y_scores))
    y_score_train = model_this_year.decision_function(X_train) 
    auc_train.append(roc_auc_score(y_train, y_score_train))

    predict_prob = model_this_year.predict_proba(X)[:, 1]
    mu = np.mean(predict)
    print(mu)
#     BS = brier_score_loss(y, predict_prob)
#     BSS = BS/(2*mu*(1-mu))
#     BSS_per_year.append(BSS)
    
#     DR_list.append(np.mean(y))
    
#     BS_per_year.append(BS)
    
#     print('Predict {} base every year model'.format(year))
    
  #

predict_results = []
EDR_list = []
BSS_2004_list = []
BS_2004_list = []
model = load(model_path) 
for year in year_list:
    feature_data_file = os.path.join(feature_data_path, '{}_log.csv'.format(year))
    target_data_file = os.path.join(target_data_path, '{}_log.csv'.format(year))
    X = genfromtxt(feature_data_file, delimiter=',')
    y = genfromtxt(target_data_file, delimiter=',')
    
    y_scores = model.decision_function(X)  
    predict_ = model.predict(X)
    predict_results.append(roc_auc_score(y, y_scores))
    
    predict_prob = model.predict_proba(X)[:, 1]
    mu = np.mean(predict_)
#     BS = brier_score_loss(y, predict_prob)
#     BSS = BS/(2*mu*(1-mu))
#     BSS_2004_list.append(BSS)
    
#     EDR_list.append(np.mean(predict_prob))
    
#     BS_2004_list.append(BS)
    print(mu)
    print('Predict {} base 2004 model'.format(year))
    

0.00038391116886185106
0.0011613350235022714
0.0004986162738854343
0.0021284491791826753
0.0012512922487670675
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.00038391116886185106
Predict 2004 base 2004 model
0.001116062641230149
Predict 2005 base 2004 model
0.00041947083358615905
Predict 2006 base 2004 model
9.823611596227734e-05
Predict 2007 base 2004 model
1.6947524814000914e-05
Predict 2008 base 2004 model
5.0638210243097165e-06
Predict 2009 base 2004 model
2.6212319790301443e-06
Predict 2010 base 2004 model
1.0461819589408453e-05
Predict 2011 base 2004 model
0.0
Predict 2012 base 2004 model
2.5578456800544308e-06
Predict 2013 base 2004 model
0.0
Predict 2014 base 2004 model
2.4642863303572967e-06
Predict 2015 base 2004 model
0.0
Predict 2016 base 2004 model
0.0
Predict 2017 base 2004 model
0.0
Predict 2018 base 2004 model


In [ ]:
# Note that even in the OO-style, we use `.pyplot.figure` to create the figure.
fig1, ax1 = plt.subplots()  # Create a figure and an axes.

ax1.plot(x_axis, DR_list, label='DR' ,marker='o')  # Plot some data on the axes.
ax1.plot(x_axis, EDR_list, label='2004 model EDR' ,marker='x')  # Plot more data on the axes...
ax1.set_xlabel('year')  # Add an x-label to the axes.
ax1.set_ylabel('DR')  # Add a y-label to the axes.
ax1.set_title("DR\EDR")  # Add a title to the axes.
ax1.legend()  # Add a legend.

fig2, ax2 = plt.subplots()

ax2.plot(x_axis, BSS_per_year, label='BSS every year' ,marker='o')  # Plot some data on the axes.
ax2.plot(x_axis, BSS_2004_list, label='BSS of 2004 model' ,marker='x')  # Plot more data on the axes...
ax2.set_xlabel('year')  # Add an x-label to the axes.
ax2.set_ylabel('BSS')  # Add a y-label to the axes.
ax2.set_title("BSS")  # Add a title to the axes.
ax2.legend()  # Add a legend.

fig3, ax3 = plt.subplots()

ax3.plot(x_axis, BS_per_year, label='BS every year' ,marker='o')  # Plot some data on the axes.
ax3.plot(x_axis, BS_2004_list, label='BS of 2004 model' ,marker='x')  # Plot more data on the axes...
ax3.set_xlabel('year')  # Add an x-label to the axes.
ax3.set_ylabel('BS')  # Add a y-label to the axes.
ax3.set_title("BS")  # Add a title to the axes.
ax3.legend()  # Add a legend.

fig4, ax4 = plt.subplots()

ax4.plot(x_axis, every_year_list, label='AUC of each year model', marker='o')  # Plot some data on the axes.
ax4.plot(x_axis, predict_results, label='AUC of 2004 model' ,marker='x')  # Plot more data on the axes...
ax4.plot(x_axis, auc_train, label='AUC of train set' ,marker='^')  # Plot more data on the axes...
ax4.set_xlabel('year')  # Add an x-label to the axes.
ax4.set_ylabel('AUC')  # Add a y-label to the axes.
ax4.set_title("AUC compare")  # Add a title to the axes.
ax4.legend()  # Add a legend.
  

fig1.savefig('./DR.jpg')
fig2.savefig('./BSS.jpg')
fig3.savefig('./BS.jpg')
fig4.savefig('./AUC_compare.jpg')

In [ ]:
a_e = np.array(every_year_list)
a_t = np.array(auc_train)
dif_ = (a_t - a_e).tolist()

fig5, ax5 = plt.subplots()

ax5.plot(x_axis, dif_, label='',marker='o')  # Plot difference
ax5.set_xlabel('year')  # Add an x-label to the axes.
ax5.set_ylabel('difference')  # Add a y-label to the axes.
ax5.set_title("AUC difference between train set and test set")  # Add a title to the axes.

fig5.savefig('./AUC_train_test_difference.jpg')

a_e = np.array(every_year_list)
a_p = np.array(predict_results)
dif = (a_e - a_p).tolist()

fig6, ax6 = plt.subplots()

ax6.plot(x_axis, dif, label='',marker='o')  # Plot difference
ax6.set_xlabel('year')  # Add an x-label to the axes.
ax6.set_ylabel('difference')  # Add a y-label to the axes.
ax6.set_title("AUC difference between 2004 and ever year model")  # Add a title to the axes.

fig6.savefig('./AUC_difference.jpg')